In [139]:
import os
import telebot 
import bs4
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt

with open("token.txt") as tk:
    token = tk.read().strip()
bot = telebot.TeleBot(token)
parsed = False
supported = ['астрономия', 'физика', 'биология', 'робототехника']

@bot.message_handler(commands=['start'])
def show_start(message):
    parsed = False
    bot.send_message(message.from_user.id, "Привет. Я предоставляю актуальные новости с ресурса https://nplus1.ru\
    по следуюшим рубрикам:  Астрономия, Физика, Биология и Робототехника. Опционально Вы можете выбрать от 1 до 30\
    статей (по умолчанию 5). После того, как данные будут собраны,\
    вы сможете: получить файл с загруженными новостями, посчитать\
    среднюю сложность выбранных статей, увидеть самую сложную и самую легкую статьи.\
    Также вы сможете получить статистику по авторам, количеству и сложности статей.\
    Чтобы посмотреть все команды напишите /help. Для начала парсинга напишите /parse.\
    Для просмотра доступных рубрик для парсинга нажмите /parse_help.")  
    
@bot.message_handler(commands=['help'])
def show_help(message):
    bot.send_message(message.from_user.id,"/parse - Ввести рубрику и запустить парсинг\n/parse_help - Вывести\
    список поддерживаемых рубрик\
    \n/file - Получить файл с данными\
    \n/mean_diff - Cредняя сложность загруженных статей.\
    \n/min_diff - Самая легкая для восприятия статья.\
    \n/max_diff - Самая сложная для восприятия статья.\
    \n/author_diff_mean - Статистика в разрезе сложности статей по авторам.\
    \n/date_diff_mean - Статистика в разрезе сложности статей по датам.\
    \n/count_by_date - Количество статей по датам.\
    \n/count_by_author - Процентное соотношение количества статей по авторам.")

@bot.message_handler(commands=['parse_help'])
def show_parse_help(message):
    bot.send_message(message.from_user.id, f"Собираю новости из рубрик:\
    Астрономия, Физика, Биология или Робототехника. Количество статей(опционально) - от 1 до 30.")
    
@bot.message_handler(commands=['parse'])
def parse(message):
    global parsed
    parsed = False
    bot.send_message(message.from_user.id, "Выберите рубрику, например, 'Робототехника'. Опционально, через пробел,\
    можете указать количество новостей(от 1 до 30), например, 'Биология 12'.")
                     
@bot.message_handler(commands=['file'])
def get_file(message):
    global parsed
    if parsed:
        fl = open('data.csv', 'rb')
        bot.send_document(message.from_user.id, fl)
        fl.close()
    else:        
        bot.send_message(message.from_user.id, "Сбор новостей не выполнен. Нажмите /parse чтобы это сделать")
                
@bot.message_handler(commands=['min_diff'])
def min_diff(message):
    global parsed
    if parsed:
        try:            
            fh = pd.read_csv('data.csv', delimiter='\t', encoding='utf-8')
            min_diff_title = fh.iloc[fh['diff'].idxmin(),1]
            min_diff_k = fh.iloc[fh['diff'].idxmin(),4]
            min_diff_url = fh.iloc[fh['diff'].idxmin(),6]        
            bot.send_message(message.from_user.id, f"Самая легкая для восприятия статья (индекс сложности: {min_diff_k})\
            \n'{min_diff_title}'.\n{min_diff_url}")            
        except Exception:
            bot.send_message(message.from_user.id, "Ошибка при анализе сложности восприятия статей.")        
    else:        
        bot.send_message(message.from_user.id, "Сбор данных не выполнен. Нажмите /parse чтобы это сделать") 

@bot.message_handler(commands=['max_diff'])
def max_diff(message):
    global parsed
    if parsed:
        try:            
            fh = pd.read_csv('data.csv', delimiter='\t', encoding='utf-8')
            max_diff_title = fh.iloc[fh['diff'].idxmax(),1]
            max_diff_k = fh.iloc[fh['diff'].idxmax(),4]
            max_diff_url = fh.iloc[fh['diff'].idxmax(),6]      
            bot.send_message(message.from_user.id, f"Самая сложная (индекс сложности: {max_diff_k}) для восприятия статья\
            '{max_diff_title}'.\n {max_diff_url}")            
        except Exception:
            bot.send_message(message.from_user.id, "Ошибка при анализе сложности восприятия статей.")        
    else:        
        bot.send_message(message.from_user.id, "Сбор данных не выполнен. Нажмите /parse чтобы это сделать") 
@bot.message_handler(commands=['mean_diff'])
def mean_diff(message):    
    global parsed
    if parsed:        
        try:            
            fh = pd.read_csv('data.csv', delimiter='\t')
            d_mean = fh["diff"].mean()
            bot.send_message(message.from_user.id, f"Средняя сложность выбранных статей: {d_mean:,.1f}")            
        except Exception:
            bot.send_message(message.from_user.id, "Ошибка при подсчете средней сложности.")        
    else:        
        bot.send_message(message.from_user.id, "Сбор новостей не выполнен. Нажмите /parse чтобы это сделать")
        
@bot.message_handler(commands=['author_diff_mean'])
def author_diff_mean(message):    
    global parsed
    if parsed:        
        try:            
            fh = pd.read_csv('data.csv', delimiter = '\t')
            by_author_count = fh.groupby('author')['diff'].count()
            by_diff_sum = fh.groupby('author')['diff'].sum()
            by_author_diff_mean_sort = (by_diff_sum / by_author_count).sort_values(ascending = False).round(1).to_dict()
            string = ""            
            for elem in by_author_diff_mean_sort:
                line = str(elem) + ": " + str(by_author_diff_mean_sort[elem])+"\n" 
                string += line 
            bot.send_message(message.from_user.id, f"Статистика в разрезе сложности статей по авторам:\n{string}")
            try:
                by_author_diff_mean_png = (by_diff_sum / by_author_count).sort_values(ascending = True).round(2)
                if len(by_author_diff_mean_png) >= 2:
                    by_author_diff_mean_png.plot(kind='bar', rot = 10)
                    plt.savefig('plot_byauthor.png', dpi = 300)
                    plt.clf()
                    bot.send_photo(message.from_user.id, photo=open('plot_byauthor.png', 'rb'))        
                    os.remove('plot_byauthor.png')
                else:
                    bot.send_message(message.from_user.id, "Для построения графика необходимо загрузить больше статей")
            except Exception:
                bot.send_message(message.from_user.id, "Не удалось построить график. Попробуйте снова.")
        except Exception:
            bot.send_message(message.from_user.id, "Ошибка при подсчете сложности.")        
    else:        
        bot.send_message(message.from_user.id, "Сбор новостей не выполнен. Нажмите /parse чтобы это сделать")
        
@bot.message_handler(commands=['date_diff_mean'])
def date_diff_mean(message):    
    global parsed
    if parsed:        
        try:            
            fh = pd.read_csv('data.csv', delimiter = '\t')            
            by_date_mean_diff = fh.groupby('date')['diff'].mean().round(1).to_dict()
            string = ''
            for elem in by_date_mean_diff:
                line = (str(elem))[:10] + ": " + str(by_date_mean_diff[elem])+"\n" 
                string += line 
            bot.send_message(message.from_user.id, f"Статистика в разрезе усреднённой сложности статей по датам:\n{string}")
            try:                
                by_date_mean_diff_png = fh.groupby('date')['diff'].mean().round(2)
                if len(by_date_mean_diff_png) >= 2:
                    by_date_mean_diff_png.plot(kind='bar', rot = 25)
                    plt.savefig('plot_bydate.png', dpi = 300)
                    plt.clf()
                    bot.send_photo(message.from_user.id, photo=open('plot_bydate.png', 'rb'))            
                    os.remove('plot_bydate.png')   
                else:
                    bot.send_message(message.from_user.id, "Для построения графика необходимо загрузить больше статей")
            except Exception:
                bot.send_message(message.from_user.id, "Не удалось построить график.")  
        except Exception:
            bot.send_message(message.from_user.id, "Ошибка при подсчете сложности по датам.")        
    else:        
        bot.send_message(message.from_user.id, "Сбор новостей не выполнен. Нажмите /parse чтобы это сделать")    

@bot.message_handler(commands=['count_by_author'])
def count_by_author(message):    
    global parsed
    if parsed:        
        try:            
            fh = pd.read_csv('data.csv', delimiter = '\t')
            by_author_count = fh.groupby('author')['author'].count()
            if len(by_author_count) >= 2:
                bot.send_message(message.from_user.id, "Процентное соотношение количества статей по авторам")
                by_author_count.plot(kind='pie', autopct = "%0.0f%%", startangle = 180)
                plt.savefig('plot_byauthor_count.png', dpi = 300)
                plt.clf()
                bot.send_photo(message.from_user.id, photo=open('plot_byauthor_count.png', 'rb'))            
                os.remove('plot_byauthor_count.png')
            else:
                bot.send_message(message.from_user.id, "Для построения графика загрузите больше статей")
        except Exception:
            bot.send_message(message.from_user.id, "Ошибка при подсчете количества статей.")        
    else:        
        bot.send_message(message.from_user.id, "Сбор новостей не выполнен. Нажмите /parse чтобы это сделать")            
@bot.message_handler(commands=['count_by_date'])
def count_by_date(message):    
    global parsed
    if parsed:        
        try:            
            fh = pd.read_csv('data.csv', delimiter = '\t')
            by_date_count = fh.groupby('date')['author'].count()            
            if len(by_date_count)>=3:
                bot.send_message(message.from_user.id, "График количества статей по датам:")
                by_date_count.plot(kind = "area", rot = 25)
                plt.savefig('plot_bydate_count.png', dpi = 300)
                plt.clf()
                bot.send_photo(message.from_user.id, photo=open('plot_bydate_count.png', 'rb'))            
                os.remove('plot_bydate_count.png')
            else:
                bot.send_message(message.from_user.id, "Для построения графика загрузите больше статей")
        except Exception:
            bot.send_message(message.from_user.id, "Ошибка при подсчете количества статей.")        
    else:        
        bot.send_message(message.from_user.id, "Сбор новостей не выполнен. Нажмите /parse чтобы это сделать")         
        
        
def ParentLink(rubric):
    """
    Processes the user's response to determine subject's parent link
    Parameters:
    accepts input as a theme selected by the user 
    """
    if rubric == 'астрономия':
        parent_link = "https://nplus1.ru/rubric/astronomy"
    elif rubric == 'физика':
        parent_link = "https://nplus1.ru/rubric/physics"
    elif rubric == 'биология':
        parent_link = "https://nplus1.ru/rubric/biology"
    elif rubric == 'робототехника':
        parent_link = "https://nplus1.ru/rubric/robots-drones"
    return parent_link

def GetLinks(orig_link, n):
    """
    Returns a list with links on the selected rubric
    also send to user topics titles loaded
    Parameters:
    parent-link is a source-link to the page with related topics(str)
    """
    page = requests.get(orig_link)
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(page.text, 'lxml')
    raw_lst = []
    i = 1
    j = 0
    for block in soup.find_all('a'):
        if j < n:
            if '/news' in block.get('href'):
                try:                    
                    link = 'https://nplus1.ru' + block.get('href')                                                         
                    raw_lst.append(link)                    
                    i += 1
                    j += 1
                except Exception:
                    bot.send_message(message.from_user.id, f"Удалось загрузить {i-1} статей" )
                    break
    return raw_lst

def GetData(link):
    """
    Returns date, title, author, description, difficulty, text of the article located at the link
    Parameters:
    link is a source-link to the topic (string)
    """    
    soup = BeautifulSoup(requests.get(link).text, 'lxml')
    
    date = (soup.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']).strip()    
    title = (soup.find_all('meta', {'property' : 'og:title'})[0].attrs['content']).strip()    
    author = (soup.find_all('meta', {'name' : 'author'})[0].attrs['content']).strip()    
    description = (soup.find_all('meta', {'name' : 'description'})[0].attrs['content']).strip()    
    diff = float((soup.find_all('span', {'class' : 'difficult-value'})[0].text).strip())
    url = (link).strip()
    
    text_raw = soup.find_all('p', {'class' : None})
    text = [t.text for t in text_raw]
    text = ' '.join(text)
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    text = (text.split("Нашли опечатку?")[0]).strip()
    
    return date, title, author, description, diff, text, url

                     
@bot.message_handler(content_types=["text"])
def get_text_messages(message):
    global parsed
    if not parsed:        
        answer = (message.text).lower().strip().split(" ")        
        try:            
            if len(answer) == 1 and answer[0] in supported:                
                parent_link = ParentLink(answer[0])
                n = 5
                bot.send_message(message.from_user.id, f" Принято! Начинаю сбор свежих новостей из раздела '{answer[0]}', {n} шт.")
            elif len (answer) == 2 and answer[0] in supported and int(answer[1]) in range(1,31):                
                parent_link = ParentLink(answer[0])
                n = int(answer[1])
                bot.send_message(message.from_user.id, f" Принято! Начинаю сбор свежих новостей из раздела '{answer[0]}', {n} шт.")
            lst = GetLinks(parent_link, n)                           
            news = []
            diff_lst = []
            for elem in lst:
                try:
                    res = GetData(elem)
                    news.append(res)
                    bot.send_message(message.from_user.id, f"Загружаю статью '{res[1]}'.")
                    diff_lst.append(res[4])
                    sleep(3)
                except Exception:
                    bot.send_message(message.from_user.id, f"Произошла ошибка при загрузке статьи '{res[1]}'.")
            global mean_d        
            mean_d = sum(diff_lst)/len(diff_lst)            
            df = pd.DataFrame(news)
            df.columns = ["date", "title", "author", "description", "diff", "text", "url"]
            df.to_csv("data.csv", sep='\t', encoding='utf-8', index=False)
            parsed = True
            bot.send_message(message.from_user.id, f"Парсинг успешно закончен. Выберите следующую команду:")
            bot.send_message(message.from_user.id, f'''/file - Получить файл с данными\
            \n/mean_diff - Cредняя сложность загруженных статей.\
            \n/min_diff - Самая легкая для восприятия статья.\
            \n/max_diff - Самая сложная для восприятия статья.\
            \n/author_diff_mean - Статистика в разрезе сложности статей по авторам.\
            \n/date_diff_mean - Статистика в разрезе сложности статей по датам.\
            \n/count_by_date - Количество статей по датам.\
            \n/count_by_author - Процентное соотношение количества статей по авторам.''')
            
        except Exception:            
            parsed = False
            bot.send_message(message.from_user.id, "Информация не распознана, сбор данных не выполнен.\
            Проверьте корректность выбранной рубрики и количество новостей(от 1 до 30). Вывести список поддерживаемых\
            рубрик /parse_help. Чтобы посмотреть все команды напишите /help")            
                             
    else:
        bot.send_message(message.from_user.id, "Команда не распознана.")
        bot.send_message(message.from_user.id, "/file - Получить файл с данными\
        \n/mean_diff - Cредняя сложность загруженных статей.\
        \n/min_diff - Самая легкая для восприятия статья.\
        \n/max_diff - Самая сложная для восприятия статья.\
        \n/author_diff_mean - Статистика в разрезе сложности статей по авторам.\
        \n/date_diff_mean - Статистика в разрезе сложности статей по датам.\
        \n/count_by_date - Количество статей по датам.\
        \n/count_by_author - Процентное соотношение количества статей по авторам.")
        bot.send_message(message.from_user.id, "Чтобы начать новый сбор данных напишите /parse")

bot.polling(none_stop = True, interval = 0)

<Figure size 432x288 with 0 Axes>